In [2]:
pip install -U selenium

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 9.4 MB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 475 kB 27.8 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 17.2 MB/s eta 0:00:01
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install bs4

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 147 kB 3.6 MB/s eta 0:00:01
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import time

# Set up the Selenium WebDriver for Safari
driver = webdriver.Safari()

In [31]:
# Using Selenium to scrape Aluminium-related news articles from a specified website, 
# loading additional articles until a target date is reached, 
# and then extracts and prints the details of each article using BeautifulSoup and save them in .json.


from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

driver = webdriver.Safari()

url = "https://news.metal.com/list/industry/aluminium"
driver.get(url)

target_date = "Jun 26, 2024"

more_news_loaded = True

while more_news_loaded:
    try:
        more_news_button = driver.find_element(By.XPATH, "//div[@class='footer___PvIjk']/span[text()='more news']")
        more_news_button.click()
        time.sleep(2)  
    except Exception as e:
        print("No more articles to load or button not found.")
        break

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    articles = soup.find_all('div', class_='newsItem___wZtKx')

    for article in articles:
        date_tag = article.find('div', class_='date___3dzkE')
        date = date_tag.text if date_tag else 'No date'

        if date > target_date:
            more_news_loaded = False
            print(f"Reached articles dated after {target_date}. Stopping scraping.")
            break

soup = BeautifulSoup(driver.page_source, 'html.parser')
articles = soup.find_all('div', class_='newsItem___wZtKx')

for article in articles:
    title_tag = article.find('div', class_='title___1baLV')
    description_tag = article.find('div', class_='description___z7ktb descriptionspec___lj3uG')
    date_tag = article.find('div', class_='date___3dzkE')
    link_tag = article.find('a', href=True)

    title = title_tag.text if title_tag else 'No title'
    description = description_tag.text if description_tag else 'No description'
    date = date_tag.text if date_tag else 'No date'
    link = link_tag['href'] if link_tag else 'No link'

    print(f"Title: {title}")
    print(f"Description: {description}")
    print(f"Date: {date}")
    print(f"Link: {link}")
    print("=" * 80)

driver.quit()


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Reached articles dated after Jun 26, 2024. Stopping scraping.
Title: Vedanta Aluminium receives BIS certification for high-quality 12 mm aluminium wire rods
Description: Vedanta Aluminium has received a certification from the Bureau of Indian Standards (BIS) for its 12 mm aluminium wire rods manufactured at the BALCO facility in Korba, India. In addition, the company has acquired six renewed certifications for other products.
Date: Aug 2, 2024 14:41
Link: https://news.metal.com/newscontent/102885969/Vedanta-Aluminium-receives-BIS-certification-for-high-quality-12-mm-aluminium-wire-rods
Title: Bauxite-rich Guinea redrafts political landscape with new constitution
Description: Guinea's military leadership has introduced a draft constitution to reduce presidential terms and limit the number of terms a president can serve the West African nation with the world's largest bauxite reserves on record. 
Date: Aug 2, 2024 14:37
Link: https://news.metal.com/newscontent/102885961/Bauxite-rich-Guin

In [50]:
# In this script we load the retrieved API key,
#initializing OpenAI client, 
#reading scraped articles from a JSON file,
# generate text embeddings for each article using the OpenAI API,
# save the embeddings to a CSV file.


load_dotenv('/Users/kesavreddy/Downloads/College/College/Actalyst/API_Key.env')
api_key = os.getenv("API_KEY")

client = OpenAI(api_key=api_key)

with open('/Users/kesavreddy/Downloads/College/College/Actalyst/scraped_articlesAluminium.json', 'r') as f:
    articles = json.load(f)

df = pd.DataFrame(articles)

def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

df['combined'] = df['title'] + ". " + df['description']
df['ada_embedding'] = df['combined'].apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))

df.to_csv('/Users/kesavreddy/Downloads/College/College/Actalyst/embedded_articles.csv', index=False)

print("Embeddings saved successfully.")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Embeddings saved successfully.


In [10]:
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from bs4 import BeautifulSoup
# import time

# # Set up the Selenium WebDriver for Safari
# driver = webdriver.Safari()

# # Open the webpage
# url = "https://news.metal.com/list/industry/aluminium"
# driver.get(url)

# # Load articles by clicking the "More News" button up to 14 times
# click_limit = 14
# click_count = 0

# while click_count < click_limit:
#     try:
#         # Find the "More News" button and click it
#         more_news_button = driver.find_element(By.XPATH, "//div[@class='footer___PvIjk']/span[text()='more news']")
#         more_news_button.click()
#         time.sleep(2)  # Wait for new articles to load
#         click_count += 1  # Increment the click count
#     except Exception as e:
#         print("No more articles to load or button not found.")
#         break

# # Use BeautifulSoup to parse the page source
# soup = BeautifulSoup(driver.page_source, 'html.parser')

# # Find all article blocks
# articles = soup.find_all('div', class_='newsItem___wZtKx')

# # Loop through each article and extract details
# for article in articles:
#     title_tag = article.find('div', class_='title___1baLV')
#     description_tag = article.find('div', class_='description___z7ktb descriptionspec___lj3uG')
#     date_tag = article.find('div', class_='date___3dzkE')
#     link_tag = article.find('a', href=True)

#     title = title_tag.text if title_tag else 'No title'
#     description = description_tag.text if description_tag else 'No description'
#     date = date_tag.text if date_tag else 'No date'
#     link = link_tag['href'] if link_tag else 'No link'

#     print(f"Title: {title}")
#     print(f"Description: {description}")
#     print(f"Date: {date}")
#     print(f"Link: {link}")
#     print("=" * 80)

# # Close the WebDriver
# driver.quit()


Title: Vedanta Aluminium receives BIS certification for high-quality 12 mm aluminium wire rods
Description: Vedanta Aluminium has received a certification from the Bureau of Indian Standards (BIS) for its 12 mm aluminium wire rods manufactured at the BALCO facility in Korba, India. In addition, the company has acquired six renewed certifications for other products.
Date: Aug 2, 2024 14:41
Link: https://news.metal.com/newscontent/102885969/Vedanta-Aluminium-receives-BIS-certification-for-high-quality-12-mm-aluminium-wire-rods
Title: Bauxite-rich Guinea redrafts political landscape with new constitution
Description: Guinea's military leadership has introduced a draft constitution to reduce presidential terms and limit the number of terms a president can serve the West African nation with the world's largest bauxite reserves on record. 
Date: Aug 2, 2024 14:37
Link: https://news.metal.com/newscontent/102885961/Bauxite-rich-Guinea-redrafts-political-landscape-with-new-constitution
Title: A

In [11]:
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from bs4 import BeautifulSoup
# import time

# # Set up the Selenium WebDriver for Safari
# driver = webdriver.Safari()

# # Open the webpage
# url = "https://news.metal.com/list/industry/aluminium"
# driver.get(url)

# # Load articles by clicking the "More News" button up to 14 times
# click_limit = 14
# click_count = 0

# while click_count < click_limit:
#     try:
#         # Find the "More News" button and click it
#         more_news_button = driver.find_element(By.XPATH, "//div[@class='footer___PvIjk']/span[text()='more news']")
#         more_news_button.click()
#         time.sleep(2)  # Wait for new articles to load
#         click_count += 1  # Increment the click count
#     except Exception as e:
#         print("No more articles to load or button not found.")
#         break

# # Use BeautifulSoup to parse the page source
# soup = BeautifulSoup(driver.page_source, 'html.parser')

# # Find all article blocks
# articles = soup.find_all('div', class_='newsItem___wZtKx')

# # List to store articles as dictionaries
# scraped_articles = []

# # Loop through each article and extract details
# for article in articles:
#     title_tag = article.find('div', class_='title___1baLV')
#     description_tag = article.find('div', class_='description___z7ktb descriptionspec___lj3uG')
#     date_tag = article.find('div', class_='date___3dzkE')
#     link_tag = article.find('a', href=True)

#     title = title_tag.text if title_tag else 'No title'
#     description = description_tag.text if description_tag else 'No description'
#     date = date_tag.text if date_tag else 'No date'
#     link = link_tag['href'] if link_tag else 'No link'

#     # Store article details in a dictionary and append to the list
#     scraped_articles.append({
#         'title': title,
#         'description': description,
#         'date': date,
#         'link': link
#     })

# # Print all scraped articles
# for article in scraped_articles:
#     print(f"Title: {article['title']}")
#     print(f"Description: {article['description']}")
#     print(f"Date: {article['date']}")
#     print(f"Link: {article['link']}")
#     #print("=" * 80)

# # Close the WebDriver
# driver.quit()


Title: Vedanta Aluminium receives BIS certification for high-quality 12 mm aluminium wire rods
Description: Vedanta Aluminium has received a certification from the Bureau of Indian Standards (BIS) for its 12 mm aluminium wire rods manufactured at the BALCO facility in Korba, India. In addition, the company has acquired six renewed certifications for other products.
Date: Aug 2, 2024 14:41
Link: https://news.metal.com/newscontent/102885969/Vedanta-Aluminium-receives-BIS-certification-for-high-quality-12-mm-aluminium-wire-rods
Title: Bauxite-rich Guinea redrafts political landscape with new constitution
Description: Guinea's military leadership has introduced a draft constitution to reduce presidential terms and limit the number of terms a president can serve the West African nation with the world's largest bauxite reserves on record. 
Date: Aug 2, 2024 14:37
Link: https://news.metal.com/newscontent/102885961/Bauxite-rich-Guinea-redrafts-political-landscape-with-new-constitution
Title: A

In [13]:
pip install numpy sentence-transformers


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 227 kB 3.3 MB/s eta 0:00:01
     |████████████████████████████████| 62.1 MB 894 kB/s eta 0:00:013
     |████████████████████████████████| 417 kB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 9.4 MB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 177 kB 4.0 MB/s eta 0:00:01
     |████████████████████████████████| 6.2 MB 3.5 MB/s eta 0:00:01
     |████████████████████████████████| 278 kB 4.1 MB/s eta 0:00:01
     |████████████████████████████████| 2.4 MB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 411 kB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 536 kB 3.1 MB/s eta 0:00:01
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install tf-keras


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.7 MB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 236.1 MB 841 kB/s eta 0:00:01     |█████                           | 36.9 MB 3.7 MB/s eta 0:00:54     |█████████████▎                  | 97.8 MB 903 kB/s eta 0:02:33
     |████████████████████████████████| 1.1 MB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 5.5 MB 763 kB/s eta 0:00:01
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.16.2
    Uninstalling tensorboard-2.16.2:
      Successfully uninstalled tensorboard-2.16.2
  Attempting uninstall: keras
    Found existing installation: keras 3.1.1
    Uninstalling keras-3.1.1:
      Successfully uninstalled keras-3.1.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.16.1
    Uninstalling tensorflow-2.16.1:
      Successfully uninstalled tensorflow-2.16.1
You should conside

In [20]:
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from bs4 import BeautifulSoup
# import time
# import json
# import os

# # Set up the Selenium WebDriver for Safari
# driver = webdriver.Safari()

# # Open the webpage
# url = "https://news.metal.com/list/industry/aluminium"
# driver.get(url)

# # Load articles by clicking the "More News" button up to 14 times
# click_limit = 14
# click_count = 0

# # List to store scraped articles
# scraped_articles = []

# while click_count < click_limit:
#     try:
#         # Find the "More News" button and click it
#         more_news_button = driver.find_element(By.XPATH, "//div[@class='footer___PvIjk']/span[text()='more news']")
#         more_news_button.click()
#         time.sleep(2)  # Wait for new articles to load
#         click_count += 1  # Increment the click count
#     except Exception as e:
#         print("No more articles to load or button not found.")
#         break

# # Use BeautifulSoup to parse the page source
# soup = BeautifulSoup(driver.page_source, 'html.parser')

# # Find all article blocks
# articles = soup.find_all('div', class_='newsItem___wZtKx')

# # Loop through each article and extract details
# for article in articles:
#     title_tag = article.find('div', class_='title___1baLV')
#     description_tag = article.find('div', class_='description___z7ktb descriptionspec___lj3uG')
#     date_tag = article.find('div', class_='date___3dzkE')
#     link_tag = article.find('a', href=True)

#     title = title_tag.text if title_tag else 'No title'
#     description = description_tag.text if description_tag else 'No description'
#     date = date_tag.text if date_tag else 'No date'
#     link = link_tag['href'] if link_tag else 'No link'

#     # Append the article to the list
#     scraped_articles.append({
#         'title': title,
#         'description': description,
#         'date': date,
#         'link': link
#     })

# # Save the scraped articles to a JSON file
# output_file = '/Users/kesavreddy/Downloads/College/College/Actalyst/scraped_articles1.json'
# with open(output_file, 'w', encoding='utf-8') as json_file:
#     json.dump(scraped_articles, json_file, ensure_ascii=False, indent=4)

# print(f"Scraped articles saved to {output_file}")

# # Close the WebDriver
# driver.quit()


Scraped articles saved to /Users/kesavreddy/Downloads/College/College/Actalyst/scraped_articles1.json


In [23]:
pip install streamlit

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 8.7 MB 3.5 MB/s eta 0:00:01
     |████████████████████████████████| 6.9 MB 14.7 MB/s eta 0:00:01
     |████████████████████████████████| 207 kB 37.7 MB/s eta 0:00:01
     |████████████████████████████████| 857 kB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 8.8 MB/s  eta 0:00:01
     |████████████████████████████████| 27.2 MB 901 kB/s eta 0:00:01
     |████████████████████████████████| 56 kB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 88 kB 5.2 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 310 kB 4.6 MB/s eta 0:00:01
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [26]:
pip install openai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 335 kB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 75 kB 18.7 MB/s eta 0:00:01
     |████████████████████████████████| 86 kB 14.4 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 20.4 MB/s eta 0:00:01
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [40]:
pip install openai


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [44]:
pip install openai python-dotenv


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [47]:
pip install openai==0.28

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 76 kB 2.9 MB/s eta 0:00:011
     |████████████████████████████████| 385 kB 10.2 MB/s eta 0:00:01
     |████████████████████████████████| 81 kB 9.6 MB/s eta 0:00:01
     |████████████████████████████████| 53 kB 10.1 MB/s eta 0:00:01
  Attempting uninstall: openai
    Found existing installation: openai 1.38.0
    Uninstalling openai-1.38.0:
      Successfully uninstalled openai-1.38.0
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import json
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv

In [7]:
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.action_chains import ActionChains
# from bs4 import BeautifulSoup
# import time

# # Set up the Selenium WebDriver for Safari
# driver = webdriver.Safari()

# # Open the webpage
# url = "https://news.metal.com/list/industry/aluminium"
# driver.get(url)

# # Scroll to the bottom of the page to load all articles
# while True:
#     # Get current height
#     last_height = driver.execute_script("return document.body.scrollHeight")
#     # Scroll down
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     # Wait for new articles to load
#     time.sleep(2)
#     # Get new height
#     new_height = driver.execute_script("return document.body.scrollHeight")
#     # Check if we have reached the bottom
#     if new_height == last_height:
#         break

# # Get page source and parse it with BeautifulSoup
# soup = BeautifulSoup(driver.page_source, 'html.parser')

# # Find all article blocks
# articles = soup.find_all('div', class_='newsItem___wZtKx')

# # Loop through each article and extract details
# for article in articles:
#     title_tag = article.find('div', class_='title___1baLV')
#     description_tag = article.find('div', class_='description___z7ktb descriptionspec___lj3uG')
#     date_tag = article.find('div', class_='date___3dzkE')
#     link_tag = article.find('a', href=True)

#     title = title_tag.text if title_tag else 'No title'
#     description = description_tag.text if description_tag else 'No description'
#     date = date_tag.text if date_tag else 'No date'
#     link = link_tag['href'] if link_tag else 'No link'

#     print(f"Title: {title}")
#     print(f"Description: {description}")
#     print(f"Date: {date}")
#     print(f"Link: {link}")
#     print("="*80)

# # Close the WebDriver
# driver.quit()


Title: Vedanta Aluminium receives BIS certification for high-quality 12 mm aluminium wire rods
Description: Vedanta Aluminium has received a certification from the Bureau of Indian Standards (BIS) for its 12 mm aluminium wire rods manufactured at the BALCO facility in Korba, India. In addition, the company has acquired six renewed certifications for other products.
Date: Aug 2, 2024 14:41
Link: https://news.metal.com/newscontent/102885969/Vedanta-Aluminium-receives-BIS-certification-for-high-quality-12-mm-aluminium-wire-rods
Title: Bauxite-rich Guinea redrafts political landscape with new constitution
Description: Guinea's military leadership has introduced a draft constitution to reduce presidential terms and limit the number of terms a president can serve the West African nation with the world's largest bauxite reserves on record. 
Date: Aug 2, 2024 14:37
Link: https://news.metal.com/newscontent/102885961/Bauxite-rich-Guinea-redrafts-political-landscape-with-new-constitution
Title: A